# Reading and Writing Files
## Introduction

This chapter is about reading files from the file system and writing to them. After finishing this lab, you can finally store data and reuse it!

*[Chapter 7](https://automatetheboringstuff.com/2e/chapter7/) in the book is about [regular expressions](https://realpython.com/regex-python/), which are a "mini-language" to search or match strings based on a pattern like `[a-z]{1,5}` (a character between a and z, between 1 and 5 times). You will learn more about regular expressions in other modules such as [Automaten und Sprachen](https://studien.rj.ost.ch/allModules/24404_M_AutoSpr.html), so we are skipping the topic in this course.*

**Instead, his notebook covers the [ninth chapter](https://automatetheboringstuff.com/2e/chapter9/) of the book.**

**Warning:** This lab will diverge from the book in a couple of important ways:

- The book describes Windows paths in most examples, while the Jupyter environment is based on Linux. Thus, paths will look like `/usr/lib` rather than e.g. `C:\\Windows\\System32`.
- In the *"Opening Files with the open() Function"* section, as well as the following sections after that: If you open a file, you will need to close it again. Otherwise, you could risk issues like **data not being fully written** to the file. Contrary to what the book shows, *don't use* `f = open(...)`. Instead, use a `with` block, which leads to Python automatically closing the file once the block is finished:

```python
with open(...) as f:
    # code that does something with 'f'
```

- Starting with the *"Saving Variables with the shelve Module"* section (including that section), we recommend you **ignore the rest of the book chapter** entirely. There are some good ways to store Python data in a file which we'll look at in later labs - unfortunately, the ways shown by the book (`shelve` and `pprint`) have various flaws. **Don't use them.**
- In addition, we go over some other good practices below. Please review them as well.
- Because of those issues in the book, we suggest that you only use the book chapter to get a rough overview of the topic, but then focus on the summary and/or optional resources below.

### Optional resources

You can find more information about reading and writing files in the Python documentation:

- [Tutorial: 7. Input and Output](https://docs.python.org/3/tutorial/inputoutput.html#tut-files)
- [Built-in Functions: `open`](https://docs.python.org/3/library/functions.html#open)
- [`pathlib` — Object-oriented filesystem paths](https://docs.python.org/3/library/pathlib.html)

Additional examples by PyMOTW (Python Module of the week):

- [pathlib — Filesystem Paths as Objects](https://pymotw.com/3/pathlib/index.html)

Relevant Real Python tutorials:

- [Reading and Writing Files in Python (Guide)](https://realpython.com/read-write-files-python/)
- [Working With Files in Python](https://realpython.com/working-with-files-in-python/) (covers some material from the next lab)
- [Python 3's pathlib Module: Taming the File System](https://realpython.com/python-pathlib/)

## Good practices

When working with files, **keep a couple of "rules" in mind**:

- Treating file paths as strings is usually a bad idea. You will need to take special care that your code works on both Windows and Linux/macOS as they use different characters to separate parts of the path. Python provides the (older) `os.path` library to do so. However, always prefer **using the `pathlib` library** which lets you handle paths in a much nicer way.
- **Be careful when deleting** files. You won't need to do so in this lab, but you will in the next lab.
- Be careful when **changing the current working directory** (e.g. via `os.chdir`). Doing so could have unintended side-effects, so it's best avoided.
- Use the provided **separate cells** to try out your code, only define the function in the first cell, without calling it. This is to avoid interfering with the tests running those cells.

## Summary

### Path Management

* Root-Folder (Anchor)
  * Folder-1 (Parent is the Root-Folder)
    * Folder-2 (Parent is Folder-1)
      * Folder-3 (Parent is Folder-2)
        * Filename.Extension
        
From the point of view of the file, all folders above it are the _parent_. The filename is also called _stem_ and the extension is the _suffix_.

#### Windows
In Windows, _backslashes_ are used to separate paths of a path: `Drive:\Root-Folder\Folder-1\Folder-2\Folder-3\Filename.Extension`. An example would be `C:\Users\harry\school\potions.xlsx`. The `C:` points to the drive, for example a hard disk or a USB stick. `.xlsx` is the file extension.

**Note that backslashes also have a special meaning in Python strings:** They are used for escape characters such as `\n`. Thus, to store a Windows path in a string, you will need to double the backslashes (`"C:\\Users"`) or use a raw string (`r"C:\Users")`. However, most of the time you should use `pathlib` instead - see below.

#### Unix-based Systems
On Linux and MacOS, _slashes_ are used to separate paths of a path: `/Folder-1/Folder-2/Folder-3/Filename.Extension`. The path starts with a slash because the first slash indicates the root folder. An example would be `/home/harry/school/potions.xlsx`.

#### Paths in Python
Thankfully, Python provides a nice abstraction over different path notations - you don't have to worry about the OS your Python app is running on. If you work with paths, use the `Path` class from the `pathlib` module:

In [2]:
from pathlib import Path

path = Path.home() / "school" / "potions"
filepath = path / "polyjuice.xlsx"

Depending on the operating system the code is run on, you'll either get a `WindowsPath` or a `PosixPath` object. Since this lab environment runs on Linux, **when we say e.g. "Return a `Path` object", you will see a `PosixPath` in the output**. 

You can join paths with a slash, thanks to slash-overloading (see the code snippet above). This method automatically applies the correct separators between the folder names. Note that the first or the second operator must be a Path-object when you use the slash to join paths.

To get the different parts of a filepath, make use of the following attributes:

In [3]:
filepath.anchor  # '/', under Windows this could be `C:\`
filepath.parts  # ('/', 'home', 'harry', 'school', 'potions', 'polyjuice.xlsx')
filepath.parent  # PosixPath('/home/harry/school/potions')
filepath.parents  # [PosixPath('/home/harry/school/potions'), PosixPath('/home/harry/school'), PosixPath('/home/harry'), PosixPath('/home'), PosixPath('/')]
filepath.name  # 'polyjuice.xlsx'
filepath.stem  # 'polyjuice'
filepath.suffix  # '.xlsx'
filepath.drive  # '', under Windows this could be 'C:'

''

You can retrieve the home directory path with `Path.home()`, and a `Path` object representing the current working directory with `Path.cwd()`.

#### Absolute and Relative Paths

Absolute and relative paths work the same as they do in a terminal. If you started Python in the `/home/harry/school/potions` directory, you can switch to any other directory with

In [ ]:
os.chdir("../../quidditch/players")

This means up two directory hierarchies and then change into the `quidditch` directory and finally into the `players` folder. This example uses a _relative path_. `..` is just a shorthand for the parent directory and the single dot `.` is a shorthand for the current directory. It also works by passing the _absolute path_:

In [ ]:
os.chdir("/home/harry/quidditch/players")  # (Absolute Path)

An *absolute path* like `/home/harry/quidditch/players` starts with the root directory, meaning with `/` on Unix platforms and with the drive letter on Windows platforms. The same absolute path always refers to the same directory.

In contrary, a *relative path* like `../../quidditch/players` or `quidditch/players` is based on the *current working directory* (`Path.cwd()` or `os.getcwd()`). This is usually the directory the script was started in (not necessarily the same as the directory the script is saved in). Relative paths can be useful when showing file paths to the user, but internally, it's often better to use absolute paths.

**Note:** While changing paths interactively (e.g. in a shell with `cd`) is done often, as outlined in the "rules" above you should avoid changing the current working directory in a script.

You can check if a path is absolute or not with the `.is_absolute()`-method on the path object:

In [4]:
relative_path = Path("quidditch", "players")
absolute_path = Path.home() / "quidditch" / "players"

relative_path.is_absolute()

False

In [5]:
absolute_path.is_absolute()

True

By using `.resolve()` on a relative path, you can turn it into an absolute one, based on the current working directory:

In [7]:
relative_path

PosixPath('quidditch/players')

In [8]:
relative_path.resolve()

PosixPath('/home/harry/school/AutPy/4-automation-basics/09-reading-and-writing-files/quidditch/players')

Or you can join it to an absolute path instead:

In [9]:
Path.home() / relative_path

PosixPath('/home/harry/quidditch/players')

Finally, by using `relative_to()`, you can get a relative path based on an absolute one:

In [11]:
absolute_path

PosixPath('/home/harry/quidditch/players')

In [12]:
absolute_path.relative_to(Path.home())

PosixPath('quidditch/players')

#### Create Folders

To create a directory in Python, you can use `os.makedirs` with the path as a string, as shown in the book. However, it's usually preferrable to use the `.mkdir()` method on a `Path` object instead:

In [ ]:
from pathlib import Path

school_path = Path.home() / "school"
transformation_path = school_path / "transformation"
november_essays_path = school_path / "divination" / "essays" / "november"

# Create a `transformation` folder in the (already existing) `school` folder
transformation_path.mkdir()

# Create a `divination` folder with an `essays` folder with a `november` folder in it
november_essays_path.mkdir(parents=True)

#### Gathering Information
The book shows how to use `os.path.getsize` to get the size of a file - again, we recommend using the pathlib methods instead.

The `.stat()` method of a `Path` object gives you [various kinds of information](https://docs.python.org/3/library/os.html#os.stat_result) about a file, including its size:

```python
potions_path = Path.home() / "school" / "potions"
polyjuice_path = potions_path / "polyjuice.xlsx"

polyjuice_path.stat().st_size
# 653364 (bytes)
```

To get the list of files in a directory, you can use `os.listdir` like shown in the book - but that gets you paths as strings again; using `.iterdir()` on the pathlib object gives you `Path` objects instead:

```python
potions_path.iterdir()
# [PosixPath('felix-felicis.xlsx'), PosixPath('love-potion.xlsx'), PosixPath('polyjuice.pptx'), PosixPath('polyjuice.xlsx')]
```

(Note the actual return value isn't a list, but an *iterable* you can e.g. use it in a `for` loop directly, or turn it into a list via `list(path.iterdir())`)

#### Path Validity
If you are working with paths on a system, you'll always have to make sure that the path actually exists. Otherwise, you'll run into errors. The following methods on the path object can help you with that:

In [ ]:
filepath.exists()  # Checks whether the path exists or not
filepath.is_file()  # Checks if the filepath points to a file or not
filepath.is_dir()  # Checks whether the path points to a directory or not

### Reading and Writing Files
To read from or write to a file, you'll first need to *open* it. You will need to specify a *mode* (e.g. "open for reading" or "open for writing"), and you will get a *file object* (sometimes called *file handler*) which lets you read/write data from the underlying file.

Depending on the operating system, there are certain constraints - for example, two different processes likely won't be able to both write to the file at the same time, as this would lead to a mix up of different data. Additionally, data is sometimes not written to the file immediately - thus, it's very important that you **take care of closing files properly**.

When you create, modify or read a file, you have to indicate in which mode you want to open it. The modes we're looking at in this chapter are the following:

  * _r_ read
  * _w_ write
  * _a_ append
  
To open a file, use the `open(...)` function. It returns a file handler. Here's a simple example on how to use `open` directly - however, this is something you should **usually not do** as you will need to close the file manually in this case:

In [ ]:
path = Path.home() / "todo.txt"

# The second argument indicates that you want to read the file. If you write to it, you'll get an error.
file = open(path, "r")
# do something with the file
file.close()

The `.open()` method on a `Path` object is equivalent:

In [ ]:
file = path.open("r")

It's **strongly recommended** to use the `with`-block when opening files. If you do that, you don't need any close-call, because that will then be done automatically. It is unfortunately not mentioned in the book, but it is best practice to use the with-block!

In [ ]:
with path.open("r") as file:
    data = file.read()

# file.close() gets called automatically

As an aside: Such a `with`-block is called a "context manager". If you want to know more about it, there's an in-depth [Real Python article](https://realpython.com/python-with-statement/) on the topic.

#### Read a File
With the `r` mode, a file handler initially points to the very beginning of the file. To read the contents of the file, use the `read()` method on the file handler.

In [ ]:
file.read()
# 'Homework\n\n- Essay on mandrakes\n- Divination chart\n- Train cup transfiguration\n'

You can see the `\n` newline-characters which are invisible in the original file:
```
Homework

- Essay on mandrakes
- Divination chart
- Train cup transfiguration
```

Based on these newlines, you can also read the file line-by-line as a list:

In [ ]:
file.seek(0)  # Resets the file handler to the beginning again

file.readlines()
# ['Homework\n', '\n', '- Essay on mandrakes\n', '- Divination chart\n', '- Train cup transfiguration\n']

However, often you don't need `.readlines()` at all, since simply iterating over a file handle will also produce individual lines:

In [ ]:
file.seek(0)  # Resets the file handle to the beginning again

for line in file:
    # ...

#### Write to a File
In order to write to a file, you have two possibilities. First, lets add a todo by opening the file in the _append_ mode:

In [ ]:
todo_path = Path.home() / "todo.txt"

with todo_path.open("a") as file:
    file.write("- Study hippogriffs")

With the `file.write()` call, you write a string to the file. This will just append it to the already existing content because you opened the file in the append-mode. But you can also overwrite a file by opening it in _w_-mode. A more common reason to open a file in the write-mode is to create one. This works the same as appending to a file, but the file does not need to exist yet:

In [ ]:
newfile_path = Path.home() / "newfile.txt"
newfile_path.exists()  # False

with newfile_path.open("w") as file:
    file.write("content of the new file")

newfile_path.exists()  # True

![The Newly Created File](images/newfile.png)

#### Quick Read/Write of Text
The `pathlib` module has two neat functions that allow you to write text to a file more quickly: `write_text()` and `read_text()`. Here's an example how it works:

In [6]:
from pathlib import Path

sample_text = """Be a nyan cat, feel great about it, 
be annoying 24/7 poop rainbows in litter box 
all day hiss at vacuum cleaner yet 
leave fur on owners clothes but catasstrophe stare out the window."""

filepath = Path("cat_ipsum.txt")

filepath.write_text(sample_text)

text = filepath.read_text()

print(text)

Be a nyan cat, feel great about it, 
be annoying 24/7 poop rainbows in litter box 
all day hiss at vacuum cleaner yet 
leave fur on owners clothes but catasstrophe stare out the window.


#### Persisting Application Data

Sometimes you want to store some data from your application in order to load it again on the next run. The book proposes two ways of doing so. Unfortunately, both ways presented in the book **have serious issues** in real-world applications, and are thus **not recommended**:

- The [shelve](https://docs.python.org/3/library/shelve.html) module (in the section *Saving Variables with the shelve Module*)
- Using [pprint.pformat](https://docs.python.org/3/library/pprint.html) (in the section *Saving Variables with the pprint.pformat() Function*)

The `shelve` module is based on [pickle](https://docs.python.org/3/library/pickle.html) - it looks like a very simple way to store/load any kind of Python objects, but don't be fooled by its simplicity - be aware of [Pickle's nine flaws](https://nedbatchelder.com/blog/202006/pickles_nine_flaws.html) before using it.

The `pprint.pformat` function is shown as another approach at data serialization in the book, which is not what this module is intended for at all - it works for most primitive data types, but this is a bad "hack" with much better solutions available.

With both approaches, among other issues, **loading stored data can run arbitrary Python code**, and thus they are unsuitable for content from an untrusted source. Imagine someone sends you data and you load it with `shelve` - it could contain malicious code that can harm your computer and you would not even notice when loading it!

Because of all those issues with the approaches presented in the book, we do not show them here, there is also no exercise using `shelve` or `pprint.pformat`. There are several better ways to persist application data, such as [JSON](https://docs.python.org/3/library/json.html). We will look at it in a later lab.

## Exercises

### Exercise 1: List Files
Implement a function called `name_and_size(directory)`: 
* `directory`: The path to a directory as a string.
* Return the files in `directory` with their corresponding file size (in Bytes) as dictionary. Only consider the files in the given directory, don't recursively go down into all subfolders.
* Do not include the file extension in the file name - check the documentation to find an elegant way how to do so.

Expected sample output (pretty-printed for readability, your function should return a dict):

```python
>>> name_and_size("exampledir")
{
    'somefile52': 871669,
    'somefile39': 4443478,
    'somefile2': 4433174,
    'somefile4104': 1533924,
}
```

In [14]:
# todo: imports
from pathlib import Path
import os

def name_and_size(directory):
    returndict = {}
    for file in Path(directory).iterdir():
        if file.is_file():
            returndict[Path(file).stem] = os.stat(file).st_size
    return returndict


Use this separate cell to try out your code.
Your code should work with the example below, but you're free to change it.

In [15]:
print(name_and_size("."))

{'reading-and-writing-files': 57242}


### Exercise 2: Grocery List
Automate your grocery shopping with Python! Write a function called `write_list(path, to_buy)` with the following properties (see the example output below):

* `to_buy`: A dictionary with items to buy (keys) and their amount each.
* Write the items to a file given in the `path` argument, which is a `Path` object
* Each line is one dictionary entry: a dash, the amount, then the item name.
* Don't worry if plural/singular don't match, e.g. `- 1 apples`; or if measure words don't match up (measure words are expressions such as "100 gram meat", "1 box of chocolate", "2 pieces of ham").
* You also don't need to worry about error handling: You can assume that the path you get is always valid and writable. Of course, for a real application, you'd need to consider scenarios such as an user passing a path to a directory which doesn't exist.
* If the file already exists, it should be overwritten.

Expected behaviour:

```python
>>> import pathlib
>>> groceries = {
    "apples": 4,
    "yoghurt": 1,
    "chocolate": 100,
    "bread": 1,
    "ham": 200,
    "eggs": 6
}
>>> out_path = pathlib.Path.cwd() / "groceries.txt"
>>> write_list(out_path, groceries)
```

This should result in a `groceries.txt` file like this:

```
- 4 apples
- 1 yoghurt
- 100 chocolate
- 1 bread
- 200 ham
- 6 eggs
```

In [43]:
import pathlib

def write_list(path, to_buy):
    file = open(path, "w")
    for key, value in to_buy.items():
        strtowrite = "- {} {}\n".format(value, key)
        file.write(strtowrite)

Use this separate cell to try out your code.
Your code should work with the example below, but you're free to change it.

In [44]:
groceries = {
    "apples": 4,
    "yoghurt": 1,
    "chocolate": 100,
    "bread": 1,
    "ham": 200,
    "eggs": 6
}


out_path = pathlib.Path.cwd() / "groceries.txt"
write_list(out_path, groceries)
print(out_path.read_text())

- 4 apples
- 1 yoghurt
- 100 chocolate
- 1 bread
- 200 ham
- 6 eggs



### Exercise 3: Create Folders and Files
This exercise aims to provide you with a sample directory structure to test your own code of the next exercise. Write a function called `create_files_and_folders(location)`. When executed, it should create the following folders, subfolders and files at the provided `location`:

```
iamroot
├── imgs
│   ├── cat.png
│   └── duck.png
└── montypython
    ├── lumberjack.txt
    └── spam.md
```

Other requirements:

* `location` is a `Path` object, representing an existing directory.
* The image files are empty.
* `spam.md` contains an excerpt from [Monty Python's spam sketch](https://en.wikipedia.org/wiki/Spam_(Monty_Python)) ([text source](http://montypython.50webs.com/scripts/Series_2/105.htm)), see below.
* `lumberjack.txt` contains an excerpt from [Monty Python's lumberjack song](https://en.wikipedia.org/wiki/The_Lumberjack_Song) ([text source](http://montypython.50webs.com/scripts/Series_1/60.htm)), see below.

Thus, if `Path("/home/lumberjack")` is given as `location`, your function should create:

- `/home/lumberjack/iamroot` (directory)
- `/home/lumberjack/iamroot/imgs` (directory)
- `/home/lumberjack/iamroot/imgs/cat.png` (empty file)
- etc.

Here are the text excerpts for you to use:

In [65]:
spam = """Scene: A cafe. One table is occupied by a group of Vikings with horned helmets on. A man and his wife enter.
Man: You sit here, dear.
Wife: All right.
Man: (to Waitress) Morning!
Waitress: Morning!
Man: Well, what've you got?
Waitress: Well, there's egg and bacon; egg sausage and bacon; egg and spam; egg bacon and spam; egg bacon sausage and spam; spam bacon sausage and spam; spam egg spam spam bacon and spam; spam sausage spam spam bacon spam tomato and spam;
Vikings: (starting to chant) Spam spam spam spam...
Waitress: ...spam spam spam egg and spam; spam spam spam spam spam spam baked beans spam spam spam...
Vikings: (singing) Spam! Lovely spam! Lovely spam!
Waitress: ...or Lobster Thermidor au Crevettes with a mornay sauce served in a Provencale manner with shallots and aubergines garnished with truffle pate, brandy and with a fried egg on top and spam.
Wife: Have you got anything without spam?
Waitress: Well, there's spam egg sausage and spam, that's not got much spam in it.
Wife: I don't want ANY spam!
Man: Why can't she have egg bacon spam and sausage?
Wife: THAT'S got spam in it!
Man: Hasn't got as much spam in it as spam egg sausage and spam, has it?
Vikings: Spam spam spam spam (crescendo through next few lines)
Wife: Could you do the egg bacon spam and sausage without the spam then?
Waitress: Urgghh!
Wife: What do you mean 'Urgghh'? I don't like spam!
Vikings: Lovely spam! Wonderful spam!
Waitress: Shut up!
"""

lumberjack = """BARBER: I'm a lumberjack, and I'm okay.
I sleep all night. I work all day.
MOUNTIES: He's a lumberjack, and he's okay.
He sleeps all night and he works all day.
BARBER: I cut down trees. I eat my lunch.
I go to the lavatory.
On Wednesdays I go shoppin'
And have buttered scones for tea.
MOUNTIES: He cuts down trees. He eats his lunch.
He goes to the lavatory.
On Wednesdays he goes shopping.
And has buttered scones for tea.
He's a lumberjack, and he's okay.
He sleeps all night and he works all day.
BARBER: I cut down trees. I skip and jump.
I like to press wild flowers.
I put on women's clothing,
And hang around in bars.
MOUNTIES: He cuts down trees. He skips and jumps.
He likes to press wild flowers.
He puts on women's clothing
And hangs around in bars?!
He's a lumberjack, and he's okay.
He sleeps all night and he works all day.
BARBER: I cut down trees. I wear high heels,
Suspendies, and a bra.
I wish I'd been a girlie,
Just like my dear Mama.
BARBER and MOUNTIES: I (He) cut(s) down trees. I (He) wear(s) high heels,
Suspendies, and a bra?!
BARBER: I wish I'd been a girlie,
Just like my dear Mama!
"""

In [99]:
from pathlib import Path, PurePath

def create_files_and_folders(location):
    root_d = location / "iamroot"
    imgs_d = root_d / "imgs"
    monty_d = root_d / "montypython"
    cat_f = imgs_d / "cat.png"
    duck_f = imgs_d / "duck.png"
    lumber_f = monty_d / "lumberjack.txt"
    spam_f = monty_d / "spam.md"
 
    root_d.mkdir()
    imgs_d.mkdir()
    monty_d.mkdir()
    cat_f.open("w")
    duck_f.open("w")
    with lumber_f.open("w") as file:
        file.write(lumberjack)
    with spam_f.open("w") as file:
        file.write(spam)
    

After writing your code, double-check that it doesn't create any files or folders outside of the given `location` - if it does, you'll need to clean up the mess yourself.

Afterwards, you can run this cell to test your code - please **don't modify it** unless you know what you're doing, as it deals with deleting data:

In [100]:
from pathlib import Path
import shutil

# Prepare a clean "data" directory in the current folder for easy testing
data_path = Path("data")
if data_path.exists():
    shutil.rmtree(data_path)
data_path.mkdir()

create_files_and_folders(data_path)
for child in data_path.iterdir():
    print(child)

data/iamroot


### Exercise 4: Find Files
To conclude this lab, you are going to write a small tool that may even help you in your daily work! People familiar with Linux might recognise this tool as an imitation of `grep`; it works as follows: The function `find_files(rootdir, extension, keyword)` takes a directory, a file extension and a keyword. It recursively searches all subfolders from the directory and checks all files of a given extension to find a specific keyword.

Note that tools such as `grep` typically accept a pattern (a ["regular expression"](https://realpython.com/regex-python/)). Since we've skipped that topic in the book, your `find_files` is much simpler: It just checks whether the given string is contained in a line, without any support for patterns.

You can use the `create_files_and_folders` function and the text excerpts from the previous exercise to check if `find_files` works the way it should.

Requirements:

* `rootdir`: A `Path` object. The path to the directory to start your search from. When `rootdir` is a non-existing directory or a file, make sure your function returns an empty list instead. Hint: You shouldn't need to check for those cases manually.
* `extension`: A string. The file extension to search for, e.g. `txt` or `png` (*without* leading dot).
* `keyword`: A string. The keyword that should be searched for in the files matching the file extension.
* The function returns a list of strings with the elements found. Format: `some/subfolder/filename.txt:linenumber:line`.
  - The path should be **relative to the given `rootdir`**
  - Lines are counted **starting at 1**
  - The trailing `\n` in lines is **stripped off**
* If binary data is read, the program should not crash, the file should just **be ignored**. You can create a file that contains binary data like so: 
`pathlib.Path("data", "program.bin").write_bytes(b"\xC0\xFF\xEE")`

Expected output (pretty-printed for readability, your function should return a list of strings):

```python
>>> find_files(Path("data", "iamroot"), "txt", "lumberjack")
[
    "montypython/lumberjack.txt:1:BARBER: I'm a lumberjack, and I'm okay.",
    "montypython/lumberjack.txt:3:MOUNTIES: He's a lumberjack, and he's okay.",
    "montypython/lumberjack.txt:13:He's a lumberjack, and he's okay.",
    "montypython/lumberjack.txt:23:He's a lumberjack, and he's okay.",
]
```

In [138]:
from pathlib import Path
 
def find_files(rootdir, extension, keyword):
    if(rootdir.exists()):
        return helper(rootdir, extension, keyword, rootdir)
    else:
        return []

def helper(rootdir, extension, keyword, acc_dir):
    occurances = []
    try:
        for item in rootdir.iterdir():
            if item.is_dir():
                occurances.extend( helper(item, extension, keyword, acc_dir) )
            elif item.is_file() and item.suffix.replace(".", "") == extension:
                with item.open("r") as file:
                    for i, line in enumerate(file.readlines()):
                        if keyword in line:
                            new_line = line.replace('\n', '')
                            occurances.append(f"{item.relative_to(acc_dir)}:{i + 1}:{new_line}")
        return occurances
    except UnicodeDecodeError:
        return []

Use this separate cell to try out your code.
Your code should work with the example below, but you're free to change it.

In [139]:
print(find_files(Path("data", "iamroot"), "txt", "lumberjack"))

["montypython/lumberjack.txt:1:BARBER: I'm a lumberjack, and I'm okay.", "montypython/lumberjack.txt:3:MOUNTIES: He's a lumberjack, and he's okay.", "montypython/lumberjack.txt:13:He's a lumberjack, and he's okay.", "montypython/lumberjack.txt:23:He's a lumberjack, and he's okay."]


# Feedback form

We'd like to get some feedback for this lab! To give us feedback, double-click the cells below and edit it in the appropriate places:

- Replace `[ ]` by `[x]` to cross checkboxes, they should look like this once you finish editing:
  * [ ] uncrossed
  * [x] crossed
- Add additional text where indicated (optional)

**Difficulty:**

The difficulty of the materials in this lab was:

- [ ] Much too difficult
- [ ] A little too difficult
- [ ] Just right
- [ ] A little too easy
- [ ] Much too easy

**Time:**

For one block (usually multiple labs), you should spend around 4h at home and 4h in the course. There are two labs in this block, so we'd expect you to spend a total of **around 4h on this one (both reading and solving)**.

For the materials in *this lab*, do you think you spent:

- [ ] Much more time
- [ ] A little more time
- [ ] About the scheduled amount of time
- [ ] A little less time
- [ ] Much less time

**Any topics you found especially enjoyable or difficult in this lab?**

<!-- Write below this line -->

**Anything else you'd like to tell us?**

<!-- Write below this line -->

# Submit

First, **save this file** (no grey dot should be visible in the tab above). Then, run the cell below to submit your work and see the results. You can submit as often as you like.

In case of problems:
- *Don't panic!*
- If you're in a course, show the error to your instructor.
- If the **tests failed** and you suspect an issue in the tests:
    * Mail your instructor, Cc `florian.bruhin@ost.ch` (if instructor != florian)
    * **No attachments** necessary.
- If the **submission failed** (error message, etc.):
    * Mail your instructor, Cc `florian.bruhin@ost.ch` (if instructor != florian)
    * Attach a screenshot of the issue
    * Attach the notebook (File > Download).

In [141]:
!submit reading-and-writing-files.ipynb

Last change: 3 seconds ago

⠏ Testing...0m
╭──────────────────────────── create folders files ────────────────────────────╮
│ 100% passed                                                                  │
╰──────────────────────────────────────────────────────────────────────────────╯
╭───────────────────────────────── find files ─────────────────────────────────╮
│ 100% passed                                                                  │
╰──────────────────────────────────────────────────────────────────────────────╯
╭──────────────────────────────── grocery list ────────────────────────────────╮
│ 100% passed                                                                  │
╰──────────────────────────────────────────────────────────────────────────────╯
╭───────────────────────────────── list files ─────────────────────────────────╮
│ 100% passed                                                                  │
╰─────────────────────────────────────────────────────────────────